In [1]:
import cv2
import numpy as np
from pathlib import Path
import csv
import sys

from ultralytics import YOLO

# --- Añadir ruta donde está BoT-SORT (ajustar a tu ruta) ---
sys.path.append("C:/Users/jpdeleon/Utec/MOT/MOT17/BoT-SORT")
from tracker.bot_sort import BoTSORT

# === Configuración ===
IMAGES_DIR = r"C:/Users/jpdeleon/Utec/MOT/MOT17/train/MOT17-02-FRCNN/img1"
MODEL_PATH = "yolov8n.pt"
OUTPUT_TXT = "motchallenge_output.txt"
ONLY_CLASS_ID = 0  # Personas

# === Definir clase Args con parámetros que necesita BoTSORT ===
class Args:
    track_high_thresh = 0.5
    track_low_thresh = 0.03
    new_track_thresh = 0.3
    track_buffer = 30
    proximity_thresh = 0.5
    appearance_thresh = 0.7
    match_thresh = 0.8
    with_reid = False
    fast_reid_config = ""
    fast_reid_weights = ""
    device = "cpu"
    cmc_method = "none"
    name = "BoTSORT"
    ablation = False
    mot20 = False

args = Args()

# --- Inicializar modelo y tracker ---
model = YOLO(MODEL_PATH)
tracker = BoTSORT(args)

# --- Procesar imágenes ---
image_files = sorted(Path(IMAGES_DIR).glob("*.jpg"))
frame_id = 0
mot_lines = []

for image_path in image_files:
    frame = cv2.imread(str(image_path))
    if frame is None:
        print(f"⚠️ No se pudo leer la imagen {image_path}")
        continue

    # Detección con YOLOv8
    results = model(frame, verbose=False)[0]

    detections_list = []
    for det in results.boxes.data.cpu().numpy():
        x1, y1, x2, y2, conf, cls = det
        if int(cls) == ONLY_CLASS_ID:
            detections_list.append([x1, y1, x2, y2, conf, cls])

    if len(detections_list) > 0:
#        detections_np = np.array(detections_list)
        detections_np = np.array(detections_list)[:, :5]  # Eliminar clase

    else:
        detections_np = np.empty((0, 6))

    # Tracking con BoTSORT
    tracks = tracker.update(detections_np, frame)

    # Guardar resultados en formato MOTChallenge
    # Formato: frame, id, x, y, w, h, -1, -1, -1, -1
    for track in tracks:
        tlwh = track.tlwh  # bbox formato tlwh
        x1, y1, w, h = tlwh
        mot_line = [
            frame_id + 1,
            track.track_id,
            int(x1),
            int(y1),
            int(w),
            int(h),
            -1, -1, -1, -1
        ]
        mot_lines.append(mot_line)

    frame_id += 1

# --- Exportar archivo MOTChallenge ---
with open(OUTPUT_TXT, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(mot_lines)

print(f"✅ Seguimiento terminado. Resultados guardados en: {OUTPUT_TXT}")


C:\ProgramData\anaconda3\envs\botsort_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: 'yolo11n.pt'

In [15]:
import cv2
import numpy as np
from pathlib import Path
import csv
import sys

from ultralytics import YOLO

# --- Añadir ruta donde está BoT-SORT (ajustar a tu ruta) ---
sys.path.append("C:/Users/jpdeleon/Utec/MOT/MOT17/BoT-SORT")
from tracker.bot_sort import BoTSORT

# === Configuración ===
IMAGES_DIR = r"C:/Users/jpdeleon/Utec/MOT/MOT17/train/MOT17-02-FRCNN/img1"
MODEL_PATH = "yolov8n.pt"
OUTPUT_TXT = "motchallenge_output.txt"
ONLY_CLASS_ID = 0  # Personas

# === Definir clase Args con parámetros que necesita BoTSORT ===
class Args:
    track_high_thresh = 0.3
    track_low_thresh = 0.05
    new_track_thresh = 0.3
    track_buffer = 30
    proximity_thresh = 0.5
    appearance_thresh = 0.7
    match_thresh = 0.8
    with_reid = False
    fast_reid_config = ""
    fast_reid_weights = ""
    device = "cpu"
    cmc_method = "none"
    name = "BoTSORT"
    ablation = False
    mot20 = False

args = Args()

# --- Inicializar modelo y tracker ---
model = YOLO(MODEL_PATH)
tracker = BoTSORT(args)

# --- Procesar imágenes ---
image_files = sorted(Path(IMAGES_DIR).glob("*.jpg"))
frame_id = 0
mot_lines = []

for image_path in image_files:
    frame = cv2.imread(str(image_path))
    if frame is None:
        print(f"⚠️ No se pudo leer la imagen {image_path}")
        continue

    # Detección con YOLOv8
    results = model(frame, verbose=False)[0]

    detections_list = []
    for det in results.boxes.data.cpu().numpy():
        x1, y1, x2, y2, conf, cls = det
        if int(cls) == ONLY_CLASS_ID:
            detections_list.append([x1, y1, x2, y2, conf, cls])

    num_detections = len(detections_list)
    print(f"🧠 Frame {frame_id+1}: {num_detections} persona(s) detectada(s)")

    if num_detections > 0:
#        detections_np = np.array(detections_list)
        detections_np = np.array(detections_list)[:, :5]  # Eliminar clase

    else:
        detections_np = np.empty((0, 6))

    # Tracking con BoTSORT
    tracks = tracker.update(detections_np, frame)
    print(f"📍 Frame {frame_id+1}: {len(tracks)} persona(s) rastreada(s)")

    # Guardar resultados en formato MOTChallenge
    # Formato: frame, id, x, y, w, h, -1, -1, -1, -1
    for track in tracks:
        tlwh = track.tlwh  # bbox formato tlwh
        x1, y1, w, h = tlwh
        mot_line = [
            frame_id + 1,
            track.track_id,
            int(x1),
            int(y1),
            int(w),
            int(h),
            -1, -1, -1, -1
        ]
        mot_lines.append(mot_line)

    frame_id += 1

# --- Exportar archivo MOTChallenge ---
with open(OUTPUT_TXT, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(mot_lines)

print(f"\n✅ Seguimiento terminado. Resultados guardados en: {OUTPUT_TXT}")


🧠 Frame 1: 7 persona(s) detectada(s)
📍 Frame 1: 6 persona(s) rastreada(s)
🧠 Frame 2: 7 persona(s) detectada(s)
📍 Frame 2: 6 persona(s) rastreada(s)
🧠 Frame 3: 6 persona(s) detectada(s)
📍 Frame 3: 6 persona(s) rastreada(s)
🧠 Frame 4: 6 persona(s) detectada(s)
📍 Frame 4: 6 persona(s) rastreada(s)
🧠 Frame 5: 6 persona(s) detectada(s)
📍 Frame 5: 6 persona(s) rastreada(s)
🧠 Frame 6: 8 persona(s) detectada(s)
📍 Frame 6: 8 persona(s) rastreada(s)
🧠 Frame 7: 11 persona(s) detectada(s)
📍 Frame 7: 8 persona(s) rastreada(s)
🧠 Frame 8: 11 persona(s) detectada(s)
📍 Frame 8: 8 persona(s) rastreada(s)
🧠 Frame 9: 12 persona(s) detectada(s)
📍 Frame 9: 9 persona(s) rastreada(s)
🧠 Frame 10: 10 persona(s) detectada(s)
📍 Frame 10: 8 persona(s) rastreada(s)
🧠 Frame 11: 12 persona(s) detectada(s)
📍 Frame 11: 10 persona(s) rastreada(s)
🧠 Frame 12: 12 persona(s) detectada(s)
📍 Frame 12: 11 persona(s) rastreada(s)
🧠 Frame 13: 11 persona(s) detectada(s)
📍 Frame 13: 9 persona(s) rastreada(s)
🧠 Frame 14: 10 person